In [1]:
!pip install nltk
!pip install pandas
!pip install numpy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install ipython-autotime

%load_ext autotime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

time: 224 ms


# Data Extraction

In [3]:
title = "20_newsgroups"

time: 403 µs


In [4]:
paths = []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/comp.graphics'):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/rec.motorcycles)'):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)

time: 3.83 ms


In [5]:
paths[0]

'/root/my_project_dir/20_newsgroups/comp.graphics/38970'

time: 9.6 ms


In [6]:
# paths.remove(paths[0]) #my computer is generating a dummy file, removing that!

time: 372 µs


In [7]:
len(paths)

1000

time: 2.62 ms


# Preprocessing

In [8]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

time: 934 µs


In [9]:
def convert_lower_case(data):
    return np.char.lower(data)

time: 968 µs


In [10]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

time: 1.15 ms


In [11]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

time: 1.62 ms


In [12]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

time: 1.13 ms


In [13]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

time: 1.48 ms


In [14]:
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

time: 2.47 ms


In [15]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

time: 1.5 ms


In [16]:
def preprocess(data, query):
    if not query:
        data = remove_header(data)        
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    data = stemming(data)
    return data

time: 1.54 ms


# Generating Postings

In [17]:
postings = pd.DataFrame()
frequency = pd.DataFrame()
doc = 0

for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)
    if doc%100 == 0:
        print(doc)

    tokens = word_tokenize(str(preprocessed_text))
    
    pos = 0
    for token in tokens:
        if token in postings:
            p = postings[token][0]            

            k = [a[0] for a in p]
            if doc in k:
                for a in p:
                    if a[0] == doc:
                        a[1].add(pos)
            else:
                p.append([doc,{pos}])
                frequency[token][0] += 1
        else:
            postings.insert(value=[[[doc, {pos}]]], loc=0, column=token)
            frequency.insert(value=[1], loc=0, column=token)

        pos += 1
    doc += 1

0
100
200
No Header
300
400
500
600
700
800
900
No Header
time: 1min 16s


In [18]:
postings

,fishkin,gorgeou,crossdo,stink,aio,wijkstra,marcel,noturn,ppmtopgm,xwdfile,...,not,electron,to,lead,or,inform,me,give,anyon,can
0,"[[999, {130, 131}]]","[[998, {174}]]","[[998, {160}]]","[[996, {260}]]","[[996, {243}]]","[[996, {242, 244}]]","[[996, {240, 241}]]","[[996, {99, 132, 125}]]","[[996, {90}]]","[[996, {86}]]",...,"[[0, {153, 42, 11}], [5, {57, 229}], [7, {1056...","[[0, {9}], [66, {61, 111}], [154, {137}], [161...","[[0, {167, 8, 104, 106, 55}], [2, {64, 27, 60,...","[[0, {6}], [70, {74}], [78, {28}], [123, {195}...","[[0, {65, 5, 113, 51, 151}], [1, {38}], [2, {2...","[[0, {4, 10, 206, 122, 191}], [5, {170, 151}],...","[[0, {128, 3, 7, 202, 78, 180}], [3, {81, 6}],...","[[0, {2, 179}], [5, {78}], [13, {150}], [41, {...","[[0, {1}], [2, {1}], [14, {37}], [18, {56}], [...","[[0, {0, 178}], [1, {44}], [2, {4}], [3, {3}],..."


time: 233 ms


In [19]:
frequency

,fishkin,gorgeou,crossdo,stink,aio,wijkstra,marcel,noturn,ppmtopgm,xwdfile,...,not,electron,to,lead,or,inform,me,give,anyon,can
0,1,1,1,1,1,1,1,1,1,1,...,379,32,814,23,497,156,346,98,276,422


time: 17.2 ms


In [20]:
postings.to_pickle(title + "_positional_postings")

time: 1.73 s


In [21]:
postings = pd.read_pickle(title + "_positional_postings")

time: 2.37 s


In [22]:
frequency.to_pickle(title + "_positional_frequency")

time: 181 ms


In [23]:
frequency = pd.read_pickle(title + "_positional_frequency")

time: 44.2 ms


# Search Query

In [24]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",frequency[preprocessed_word][0])
    print("Postings List:",postings[preprocessed_word][0])

time: 37.5 ms


In [25]:
def get_positions(posting_values, doc):
    for posting_value in posting_values:
        if posting_value[0] == doc:
            return posting_value[1]
    return {}

time: 283 ms


In [26]:
def gen_init_set_matchings(word):
    init = []
    word_postings = postings[word][0]
    for word_posting in word_postings:
        for positions in word_posting[1]:
            init.append((word_posting[0], positions))
    return init

time: 25.7 ms


In [27]:
def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]

        count = 0

        for k in b:
            pos = pos+1
            k_pos = postings[k][0]
            docs_list = [z[0] for z in k_pos]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)

time: 4.05 ms


In [28]:
def run_query(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        print("Total Document Mathces", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(init_word)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    return total_matched_docs

time: 3.84 ms


In [29]:
def print_file(file):
    out_file = open(paths[file], 'r', encoding='cp1250')
    out_text = out_file.read()
    print(out_text)

time: 2.69 ms


In [30]:
get_word_postings("welcome")

welcom
Frequency: 24
Postings List: [[38, {779, 270}], [141, {6624, 26, 6699}], [167, {168}], [186, {993}], [205, {61}], [223, {213}], [388, {185}], [435, {408, 174}], [441, {204}], [480, {69}], [504, {65}], [529, {196}], [544, {88}], [548, {91}], [603, {281}], [649, {26, 6508, 6583}], [671, {435}], [706, {21}], [749, {3109}], [823, {40}], [890, {173}], [956, {25, 6586, 6661}], [978, {631}], [986, {113}]]
time: 9.66 ms


In [31]:
query = "not in"

time: 1.84 ms


In [32]:
lists = run_query(query)

not in
['not', 'in']
Total Document Matches: {768, 129, 774, 649, 141, 529, 277, 795, 805, 173, 562, 824, 314, 956, 830, 966, 327, 341, 992, 485, 742, 743, 869, 748, 749, 623, 245}
time: 111 ms


In [33]:
print_file(768)

Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:38476 comp.sys.amiga.graphics:14031 comp.graphics.animation:3025
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!usc!zyeh
From: zyeh@caspian.usc.edu (zhenghao yeh)
Newsgroups: comp.graphics,comp.sys.amiga.graphics,comp.graphics.animation
Subject: Re: Newsgroup Split
Date: 19 Apr 1993 15:58:31 -0700
Organization: University of Southern California, Los Angeles, CA
Lines: 18
Sender: zyeh@caspian.usc.edu (zhenghao yeh)
Distribution: world
Message-ID: <lt6bknINNb6l@caspian.usc.edu>
References: <C5q0HK.KoD@hawnews.watson.ibm.com> <1993Apr19.144533.6779@cs.ruu.nl> <1993Apr19.193758.12091@unocal.com> <1quvdoINN3e7@srvr1.engin.umich.edu>
NNTP-Posting-Host: caspian.usc.edu


In article <1quvdoINN3e7@srvr1.engin.umich.edu>, tdawson@engin.umich.edu (Chris Herringshaw) writes:
|> Concerning the proposed newsgroup split, I personally am not in favor of
|> doing this.  I learn an awful lot about all aspects of grap